### Další koncepty objektově-orientovaného programování
---

<br>

#### Podtržítka v Pythonu

---
Podtržítko je **syntaktický znak** v Pythonu, který má nejeden důležitý význam:
1. Přeskoč `_`,
2. privátní proměnná  `_name`,
3. chráněná proměnná `__name`,
4. přepis klíčové slovo `class_`,
5. magická metoda `__init__`.

<br>

##### Samotné podtržítko (*skip*)

In [1]:
import time

def check_logging_messages(limit: int) -> None:
    """Check the current logging messages."""
    
    for sec in range(limit):
        print("Checking logging messages..")
        time.sleep(1)
        
check_logging_messages(5)

Checking logging messages..
Checking logging messages..
Checking logging messages..
Checking logging messages..
Checking logging messages..


<br>

Proměnná `sec`, kterou jsme vytvořili v rámci funkce `check_logging_messages` nemá žádné využití.

<br>

Proto v Pythonu existuje symbol `_` pro označení **nepoužívané** proměnné:
```python
    for _ in range(limit):
        print("Checking logging messages..")
```

<br>

Případně je možné **přeskakovat** i u *vícenásobného* přiřazování:
```python
jmeno, domena = "matous@gmail.com".split("@")

# nepotřebuji proměnnou jméno
_, domena = "matous@gmail.com".split("@")
print(domena)
```

<br>

##### Privátní objekty (*weak private*)

Některé jazyky umožňují práci s pomocí **privátních** (soukromých) proměnných (~Java).

<br>

Python tuto funkcionalitu **nepodporuje**:

In [2]:
import time

class ProtocolChecker:
    
    def __init__(self, limit: int, protocol: str):
        self._limit = limit
        self.protocol = protocol
        
    def check_protocol(self) -> None:
        for _ in range(self._limit):
            print(f"Checking routing protocol:{self.protocol}...")
            time.sleep(1)

In [3]:
checker = ProtocolChecker(5, "ospf")
print(checker._limit)

5


In [4]:
checker._limit = 10
print(checker._limit)

10


<br>

Pokud metodu nebo proměnnou označíš pomocí **jednoho podtržítka**, naznačuješ ostatním programátorům, že tento objekt je **interní** a není *DOPORUČENÉ* jej přepisovat.

<br>

##### Chráněné proměnné (*strong private*)

Pomocí **dvou podtržítek** může uživatel definovat **chráněné** proměnné, a tím předejít neúmyslnému přepsání nebo přetypování:

In [5]:
import time

class ProtocolChecker:
    
    def __init__(self, limit: int, protocol: str):
        self._limit = limit
        self.protocol = protocol
        
    def check_protocol(self) -> None:
        for _ in range(self._limit):
            print(f"Checking routing protocol:{self.protocol}...")
            time.sleep(1)

In [6]:
checker = ProtocolChecker(5, "bgp")

In [7]:
print(f"{checker.__limit=}")

AttributeError: 'ProtocolChecker' object has no attribute '__limit'

In [3]:
print(checker.__dict__)

{'_ProtocolChecker__limit': 5}


In [5]:
checker._ProtocolChecker__limit = 10
print(checker.__dict__)

{'_ProtocolChecker__limit': 10}


<br>

Pomocí metody `__dict__` je možné zjistit, jaké **proměnné** (..a hodnoty) má instance k dispozici a dohledáme přejmenovanou proměnnou.

<br>

##### Vlastní jméno (*any*)

Pokud se ti bude krýt **klíčové slovo** se jménem proměnné, interpret ti bude vracet **syntaktickou výjimku**:

In [10]:
class Employer:
    def __init__(self, name: str, email: str):
        self.name = name
        self.email = email

class = Employer("Matous", "matous@gmail.com")  # rezervovaný výraz

SyntaxError: invalid syntax (<ipython-input-10-9e5025b3f93d>, line 6)

<br>

Pokud tomu chceš zabránit, můžeš použít podtržítko jako příponu za klíčovým výrazem:
```python
class_ = Employer("Matous", "matous@gmail.com")
```

<br>

##### Magické metody (double-underscore methods ~ dunder methods)

Magické metody jsou **speciální metody** v Pythonu, na kterých stojí veškerá práce s objekty.

<br>

Oficiální dokumentace ale není příliš nápomocná a jejich používání není úplně nejčastější (výjimky typu `__init__` aj.).

<br>

#### Vlastnosti třídy

---
Kdy a kde používat privátní a chráněné proměnné a proč?

<br>

Představ si situaci, kdy máš napsat převodník jednotek objemu. Z **litrů** na **pinty**(UK):

In [8]:
class LiterConvertor:
    def __init__(self, liter: int = 0):
        self.liter = liter
        self.pint_uk_coef: float = 1.759754
        self.to_pints: float = self.liter * self.pint_uk_coef

In [9]:
bottle_volume = LiterConvertor()
bottle_volume.liter = 0.75
print(f"{bottle_volume.liter=}")

bottle_volume.liter=0.75


<br>

Pokud hodnotu nastavíš jako defaultní, s tím že ji budeš měnit později, nastane tato situace:

In [3]:
print(f"{bottle_volume.to_pints=}")

bottle_volume.pints_uk=0.0


<br>

Interpret použil pro vytvoření hodnoty v proměnné `bottle_volume.pints_uk` původní **defaultní** hodnotu `0` a na přepsání nereaguje.

<br>

Celý proces můžeš zkontrolovat magickou metodou `__dict__`:

In [40]:
 print(bottle_volume.__dict__)

{'liter': 0.75, 'pint_uk_coef': 1.759754, 'pints_uk': 0.0}


<br>

Kdykoliv se pokusíš tímto způsobem přepsat hodnotu, interpret pracuje stejně jako když přepisuješ hodnotu ve slovníku.

In [6]:
print(
    f"{bottle_volume.liter=}",              # instační atribut
    f"{bottle_volume.__dict__['liter']=}",  # slovníkový výběr podle klíče
    sep="\n"
)

bottle_volume.liter=0.75
bottle_volume.__dict__['liter']=0.75


<br>

Programátoři z jiných jazyků, např. Java, by napsali jiné řešení:

In [10]:
class LiterConvertor:

    def __init__(self, liter: int = 0):
        self.set_volume_in_liter(liter)
        self.pint_uk_coef: float = 1.759754
        
    def to_pints(self):
        return self._liter * self.pint_uk_coef
    
    def get_volume_in_liter(self):
        return self._liter
    
    def set_volume_in_liter(self, value: float):
        if value < 0:        # doplňující podmínka
            raise ValueError("Cannot process negative number")
        self._liter = value  # privátní proměnná

In [13]:
bottle_volume_1 = LiterConvertor()

In [14]:
bottle_volume_2 = LiterConvertor(0.75)
print(
    f"{volume.get_volume_in_liter()=}",  # pro hodnotu musím použít metodu
    f"{volume.to_pints()=}",
    sep="\n"
)

volume.get_volume_in_liter()=0.75
volume.to_pints()=1.3198155


<br>

V tento moment pracujeme s **privátní** proměnnou, tak jak v jiných jazycích.

In [15]:
volume.set_volume_in_liter(-1)

ValueError: Cannot process negative number

<br>

Náš předpis má umožnit efektivní **přepsání** hodnoty ve stávajícím zápisu.

<br>

ale v tento moment bychom provedli příliš mnoho zásahů do původního zápisu.

<br>

U každého nového spuštění, bych potřeboval doplnit **kulatou** závorku.

<br>

Také se objevil problém s ohledem na implementaci našeho vylepšení.

<br>

Všechny ohlášení `obj.liter` je nutné přepsat na `obj.get_volume_in_liter()` a `obj.liter = val` na `obj.set_volume(val)`.

<br>

Taková úprava řešení může znamenat problémy na desítky, stovky řádků, protože přidáme nové metody, které je potřeba použít.

<br>

Naštěstí můžeme s vlastnostmi třídy nakládat lépe pomocí funkce `property`:

In [16]:
class LiterConvertor:
    def __init__(self, liter: float = 0.0):
        self.liter = liter
        self.pint_uk_coef: float = 1.759754
        
    def to_pints(self):
        return self._liter * self.pint_uk_coef
    
    def get_volume_in_liter(self):
        print("Getting..")
        return self._liter
    
    def set_volume_in_liter(self, value: float):
        print("Setting..")
        if value < 0:
            raise ValueError("Cannot process negative number")
        self._liter = value
        
    liter = property(get_volume_in_liter, set_volume_in_liter)

In [19]:
volume_1 = LiterConvertor()
print(f"{volume_1.liter=}")

Setting..
Getting..
volume_1.liter=0.0


In [21]:
volume_2 = LiterConvertor(0.75)
print(
    f"{volume_2.liter=}",  # opět stačí aplikovat jen atribut metody (privátní atribut)
    f"{volume_2.to_pints()=}",
    sep="\n"
)

Setting..
Getting..
volume_2.liter=0.75
volume_2.to_pints()=1.3198155


<br>

Pokaždé co tentokrát použijeme proměnnou `liter` dojde automaticky k zavolání metody `get_volume_in_liter()`.

<br>

Stejně tak, pokud budeš chtít přepsat hodnotu v proměnné `liter`, tak dojde ke spuštění metody `set_volume_in_liter()`.

<br>

Takže pokud použiješ funkci `property()`, nemusíš se omezovat na potřebné úpravy tvého stávajícího skriptu na tolika místech.

<br>


#### Dekorátory setter, getter, deleter

---

<br>

Zabudovaná funkce `property` při spuštění vytváří objekt `property`. V dokumentaci uvidíš syntaxi:
```python
property(fget=None, fset=None, fdel=None, doc=None)
```

<br>

V předchozí ukázce jsme zapisovali ručně metody `set_volume` a `get_volume`, které odpovídají volitelným argumentém u funkce `property`.

<br>

Zkušenější programátoři s těmito atributy pracují pomocí **dekorátorů** `@property` a `@setter`.

<br>

Pokud **neznáš dekorátory**, nic si z toho nedělej. Pro skutečné použití tohoto řešení stačí pracovat s jejich abstrakcemi.

<br>

Původní zápis pomocí dekorátorů upravíme následovně:

In [22]:
class LiterConvertor:
    def __init__(self, liter: float = 0.0):
        self.liter = liter
        self.pint_uk_coef: float = 1.759754
        
    def to_pints(self):
        return self._liter * self.pint_uk_coef
    
    @property         # označíme jako vlastnost pomocí dekorátoru
    def liter(self):  # jméno metody nezačíná slovesem
        print("Getting..")
        return self._liter
    
    @liter.setter     # dekorátor pro nastavení hodnoty 
    def liter(self, value: float):  # stejnojmenná metoda
        print("Setting..")
        if value < 0:
            raise ValueError("Cannot process negative number")
        self._liter = value

In [24]:
volume = LiterConvertor(0.75)
print(volume.liter)
print(f"{volume.to_pints()=:.2f}")

Setting..
Getting..
0.75
volume.to_pints()=1.32


<br>

Dnes jde o nejrozšířenější implementaci nastavení **vlastostni třídy**, která vyplývá z funkce `property`. Výsledné řešení je poměrně jednoduché, přehledné a dá se pochopit bez podrobné znalosti dekorátorů.

<br>

#### Metody v OOP

---

Jako ucelený seznam metod, které můžeme použít v rámci OOP se podívej na tyto metody:
1. **Instanční** metoda,
2. **třídní** metoda,
3. **statická** metoda.

<br>

Celou situaci lze vysvětlit na teoretické ukázce:

In [ ]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    
    def load_data(self):     # klasická metoda instance
        return "Calling instance method..", self
    
    @classmethod
    def parsing_files(cls):  # třídní metoda
        return "Calling class method..", cls
    
    @staticmethod
    def is_there_file():     # statická metoda
        return "Calling static method.."

In [ ]:
reader = KindleNotesParser()

<br>

##### Instační metoda
Metoda, která má mj. mezi parametry klíčový výraz `self`. Tím dovede zpřístupnit jak třídy (a její atributy), tak instance.

In [ ]:
print(
    reader,              # adresa objektu instance 'reader'
    reader.load_data,    # popis metody patřící instanci 'reader'
    reader.load_data(),  # spuštění, opět adresa
    sep="\n"
)

<br>

Takže pomocí předchozí ukázky můžeme říct, že **instanční metoda** je schopna zpřístupnit *původní třídu* a případně pracovat s jejími *atributy*.

<br>

##### Třídní metoda
Podobný zápis jako pro instační metodu, ale liší se dvěma zásadním rozdíly:
1. Dekorátor `@classmethod`,
2. parametr `cls`.

In [ ]:
print(
    reader.parsing_files,    # adresa objektu instance nyní chybí
    reader.parsing_files(),  # metodu spouští, třídy zpřístupní, ale instanci nezná
    sep="\n"
)

<br>

Jakmile použiješ **třídní metodu**, vidíš že máš přístup k objektu *původní třídy*, ale tentokrát není k dispozici odkaz (adresa) *instance*.

```
('Reading .txt file..', <__main__.KindleParser object at 0x7f0ed30782e0>)
('Spoustim metodu tridy', <class '__main__.KindleParser'>)
```

<br>

##### Statická metoda

1. Dekorátor `@staticmethod`,
2. chybí parametr `self`,
3. chybí parametr `cls`.

In [ ]:
print(
    reader.is_there_file,
    reader.is_there_file(),
    sep="\n"
)

In [ ]:
print(KindleNotesParser.load_data())

In [ ]:
print(KindleNotesParser.parsing_files())

In [ ]:
print(KindleNotesParser.is_there_file())

<br>

Pokud spustíš a prozkoumáš **statickou metodu**, můžeš si ověřit, že tato metoda nemá přístup ani k *původní třídě*, ani k její *instanci*.

<br>

##### Shrnutí k metodám

---
1. **instanční metoda** - může upravit nejenom objekty instance, ale i třídy (na začátku vidí jak třídu, tak instanci),
2. **třídní metoda (@classmethod)** - může upravit objekty třídy, ale nemůže upravovat objekty instancí (vidí třídu, ale ne instanci),
3. **statická metoda (@staticmethod)** - nemůže upravovat ani objekty instancí, ani objekty třídy (nevidí ani třídu, ani instanci).

<br>

##### Praktické ukázky

---

###### Instanční metoda

In [ ]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    notes: list = []  # upravení třídního atributu
    
    def __init__(self, file: str, data: str):
        self.file = file
        self.data = data
#         self.notes = list()  # upravení instance atributu
        
    def load_data(self):
        return self.notes.append(self.data)

In [ ]:
first_note = KindleNotesParser("poznamky.txt", "Moje první poznámka k ...")
second_note = KindleNotesParser("poznamky_nove.txt", "Druhá poznámka ke knížce ...")

first_note.load_data()
second_note.load_data()

print(first_note.notes, second_note.notes, sep="\n")

<br>

Je jedno, který atribut budeš chtít upravit. Díky **instanční metodě** můžeš pracovat jak s třídními, tak s instančními objekty.
<br>

###### Třídní metoda

In [ ]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    readed_files: int = 0
    
    def __init__(self, file: str):
        self.file = file
    
    @classmethod
    def parsing_files(cls, name):
        instance = cls(name)
        cls.readed_files += 1  
        print(f"Parsing data from: {name}")

In [ ]:
print(f"{KindleNotesParser.readed_files=}")

In [ ]:
KindleNotesParser.parsing_files("poznamky.txt")
KindleNotesParser.parsing_files("nove_poznamky.txt")
KindleNotesParser.parsing_files("poznpozn.txt")

In [ ]:
print(f"{KindleNotesParser.readed_files=}")

<br>

Pokud budeš chtít použít **třídní metody**, potom dávej pozor na to, že můžeš spravovat pouze třídní atributy.
<br>

###### Statická metoda

In [ ]:
import os

class KindleNotesParser:
    """Parse the data in the .txt file"""
    
    def __init__(self, file: str):
        self.file = file
    
    @staticmethod
    def is_there_file(name):
        print("The file exists!") if os.path.exists(name) else print("Does not exist!")


In [ ]:
parser = KindleNotesParser("")
parser.is_there_file("lesson01.ipynb")

In [ ]:
parser.is_there_file("lesson11.ipynb")

<br>

**Statická metoda** nepotřebuje vědět nic ani o třídě, ani o instanci. Pracuje s parametrem jako klasická funkce. Ale svým účelem spadá jako nástroj ke konkrétní třídě.

<br>

#### Úloha
----

První úlohou bude napsat službu zpracovávající firemní tickety.

Průběh celé aplikace:
```
"""
1. Iniciace aplikace (zadáme škálu důležitosti priorit),
2. zadáme několik nových ticketů (jméno, zpráva),
3. zpracujeme tickety,
"""
```


Následný výstup:
```
Created: 09/11/21, 08:37:09
Processing ticket id: TPTUZSIY
Issue: ...
Customer: ...
Importance: 9
==================================
Created: 09/11/21, 08:37:09
Processing ticket id: DOGGOKZL
Issue: ...
Customer: ...
Importance: 10
==================================
Created: 09/11/21, 08:37:09
Processing ticket id: LUTAIXAS
Issue: ...
Customer: ...
Importance: 4
==================================
```

In [25]:
note = \
"""
==========
Faktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová)
- Your Highlight on Location 2724-2728 | Added on Wednesday, July 24, 2019 8:41:27 AM

Největším orgánem našeho těla je kůže. Před objevem moderních léků patřila k nejhorším kožním nemocem syfilis. Začínala jako svědivé vřídky a pak si prokousala cestu do kostí, až postihla celou kostru. Nemoc způsobující ohavný vzhled a nesnesitelnou bolest měla v různých zemích různá jména. V Rusku chorobě říkali „polská nemoc“. V Polsku to byla „německá nemoc“, v Německu „francouzská nemoc“ a ve Francii „italská nemoc“. Italové vinu házeli zpátky a nazývali ji „francouzská nemoc“.
==========
Life Is What You Make It (Peter Buffett)
- Your Highlight on Location 233-234 | Added on Monday, August 19, 2019 2:07:48 PM

The problem with honoring the rewards of work rather than the work itself is that the rewards can always be taken away.
==========
"""

In [34]:
note_atributes = note.split("=========="); note_atributes
notes = [note for note in note.split("==========") if note != "\n"]; notes

['\nFaktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová)\n- Your Highlight on Location 2724-2728 | Added on Wednesday, July 24, 2019 8:41:27 AM\n\nNejvětším orgánem našeho těla je kůže. Před objevem moderních léků patřila k\xa0nejhorším kožním nemocem syfilis. Začínala jako svědivé vřídky a\xa0pak si prokousala cestu do kostí, až postihla celou kostru. Nemoc způsobující ohavný vzhled a\xa0nesnesitelnou bolest měla v\xa0různých zemích různá jména. V\xa0Rusku chorobě říkali „polská nemoc“. V\xa0Polsku to byla „německá nemoc“, v\xa0Německu „francouzská nemoc“ a\xa0ve Francii „italská nemoc“. Italové vinu házeli zpátky a\xa0nazývali ji „francouzská nemoc“.\n',
 '\nLife Is What You Make It (Peter Buffett)\n- Your Highlight on Location 233-234 | Added on Monday, August 19, 2019 2:07:48 PM\n\nThe problem with honoring the rewards of work rather than the work itself is that the rewards can always be taken away.\n']

In [37]:
clean_note: list = [item for item in notes[0].splitlines() if item]

In [38]:
clean_note

['Faktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová)',
 '- Your Highlight on Location 2724-2728 | Added on Wednesday, July 24, 2019 8:41:27 AM',
 'Největším orgánem našeho těla je kůže. Před objevem moderních léků patřila k\xa0nejhorším kožním nemocem syfilis. Začínala jako svědivé vřídky a\xa0pak si prokousala cestu do kostí, až postihla celou kostru. Nemoc způsobující ohavný vzhled a\xa0nesnesitelnou bolest měla v\xa0různých zemích různá jména. V\xa0Rusku chorobě říkali „polská nemoc“. V\xa0Polsku to byla „německá nemoc“, v\xa0Německu „francouzská nemoc“ a\xa0ve Francii „italská nemoc“. Italové vinu házeli zpátky a\xa0nazývali ji „francouzská nemoc“.']

In [42]:
title, author = clean_note[0].split(" (", maxsplit=1)  # titulek (autor) 

In [43]:
title

'Faktomluva'

In [45]:
author.rstrip(")")

'Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová'

In [46]:
location, datetime = clean_note[1].split("|")  # umístění | datum, čas

In [51]:
location.strip()

'- Your Highlight on Location 2724-2728'

In [52]:
datetime.strip()

'Added on Wednesday, July 24, 2019 8:41:27 AM'

In [53]:
clean_note[2]  # poznámka

'Největším orgánem našeho těla je kůže. Před objevem moderních léků patřila k\xa0nejhorším kožním nemocem syfilis. Začínala jako svědivé vřídky a\xa0pak si prokousala cestu do kostí, až postihla celou kostru. Nemoc způsobující ohavný vzhled a\xa0nesnesitelnou bolest měla v\xa0různých zemích různá jména. V\xa0Rusku chorobě říkali „polská nemoc“. V\xa0Polsku to byla „německá nemoc“, v\xa0Německu „francouzská nemoc“ a\xa0ve Francii „italská nemoc“. Italové vinu házeli zpátky a\xa0nazývali ji „francouzská nemoc“.'

In [116]:
class ClippingParser:
    """Parse the text with clippings."""
    
    def __init__(self, text: str, separator: str):
        self.text = text
        self.separator = separator
        
    def split_notes(self) -> list:
        return self.text.split(self.separator)
    
    def filter_simple_newlines(self, clippings: list) -> list:
        return [
            clipping
            for clipping in clippings
            if clipping != "\n" and clipping != ""
        ]

    def parse_specific_data(self, line: list, sep: str = "") -> tuple:
        """Select specific index in the clean clippings and parse the values."""
        try:
            splitted_line: list = line.split(sep, maxsplit=1)

        except AttributeError:
            print("Error, huh!")
            before = None
            after = None
        else:
            before, after = splitted_line
        finally:
            return before.strip(), after.strip()

        
class ClippingProcessor:
    def __init__(self):
        self.list_of_notes: list = []

    def process_note(self, notes: KindleNote, *args):
        self.list_of_notes.append(
            notes(
                title=args[0],
                author=args[1],
                location=args[2],
                datetime=args[3],
                note=args[4]   
            )
        )
        

In [118]:
clips = ClippingParser(
    """
==========
Faktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová)
- Your Highlight on Location 2724-2728 | Added on Wednesday, July 24, 2019 8:41:27 AM

Největším orgánem našeho těla je kůže. Před objevem moderních léků patřila k nejhorším kožním nemocem syfilis. Začínala jako svědivé vřídky a pak si prokousala cestu do kostí, až postihla celou kostru. Nemoc způsobující ohavný vzhled a nesnesitelnou bolest měla v různých zemích různá jména. V Rusku chorobě říkali „polská nemoc“. V Polsku to byla „německá nemoc“, v Německu „francouzská nemoc“ a ve Francii „italská nemoc“. Italové vinu házeli zpátky a nazývali ji „francouzská nemoc“.
==========
Life Is What You Make It (Peter Buffett)
- Your Highlight on Location 233-234 | Added on Monday, August 19, 2019 2:07:48 PM

The problem with honoring the rewards of work rather than the work itself is that the rewards can always be taken away.
==========
""",
"==========\n"
)

cc = clips.filter_simple_newlines(clips.split_notes())
data = tuple(
    (
        clips.parse_specific_data(cc[0].splitlines()[0], " ("),
        clips.parse_specific_data(cc[0].splitlines()[1], "|"),
        cc[0].splitlines()[3]
    )
)
ClippingProcessor().process_note(KindleNote, data)

IndexError: tuple index out of range

In [108]:
class KindleNote:
    """Create a new book note"""
    
    def __init__(self, title: str, author: str,
                 location: str, datetime: str, note: str):
        self.note = note
        self.title = title
        self.author = author
        self.location = location
        self.datetime = datetime
        self.note = note
        
note1 = KindleNote(
    title=title,
    author=author,
    location=location,
    datetime=datetime,
    note=note
)

In [109]:
note1.datetime

'Added on Wednesday, July 24, 2019 8:41:27 AM'

In [24]:
tuple(filter(None, note_atributes))

('Faktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová)',
 '- Your Highlight on Location 2724-2728 | Added on Wednesday, July 24, 2019 8:41:27 AM',
 'Největším orgánem našeho těla je kůže. Před objevem moderních léků patřila k\xa0nejhorším kožním nemocem syfilis. Začínala jako svědivé vřídky a\xa0pak si prokousala cestu do kostí, až postihla celou kostru. Nemoc způsobující ohavný vzhled a\xa0nesnesitelnou bolest měla v\xa0různých zemích různá jména. V\xa0Rusku chorobě říkali „polská nemoc“. V\xa0Polsku to byla „německá nemoc“, v\xa0Německu „francouzská nemoc“ a\xa0ve Francii „italská nemoc“. Italové vinu házeli zpátky a\xa0nazývali ji „francouzská nemoc“.')

In [20]:
note = dict()

note["name"] = note_atributes[0]
note["location"] = note_atributes[1].split("|")[0]
note["date"] = note_atributes[1].split("|")[1]
note["description"] = note_atributes[-1]

In [22]:
from pprint import pprint
pprint(note)

{'date': ' Added on Wednesday, July 24, 2019 8:41:27 AM',
 'description': 'Největším orgánem našeho těla je kůže. Před objevem moderních '
                'léků patřila k\xa0nejhorším kožním nemocem syfilis. Začínala '
                'jako svědivé vřídky a\xa0pak si prokousala cestu do kostí, až '
                'postihla celou kostru. Nemoc způsobující ohavný vzhled a\xa0'
                'nesnesitelnou bolest měla v\xa0různých zemích různá jména. '
                'V\xa0Rusku chorobě říkali „polská nemoc“. V\xa0Polsku to byla '
                '„německá nemoc“, v\xa0Německu „francouzská nemoc“ a\xa0ve '
                'Francii „italská nemoc“. Italové vinu házeli zpátky a\xa0'
                'nazývali ji „francouzská nemoc“.',
 'location': '- Your Highlight on Location 2724-2728 ',
 'name': 'Faktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová)'}


In [64]:
from typing import List


class KindleNoteParser:
    """Process the .txt file and create new notes."""
    
    def __init__(self, file = None):
        self.file = file
        self.notes: list = []
        self.separator = "==========\n"
            
    def process_txt_content(self, txt_cont: List[str]) -> List[str]:
        """Split the .txt file into individual notes."""
        return txt_cont.split(self.separator)
    
    def split_text_into_lines(self, all_notes: List[str]):
        """Split each note into individual attributes, process them and return all results."""
        return [
            self.process_note_content(note.splitlines())
            for note in all_notes
            if note
        ]
                
    def process_note_content(self, note: List[str]):
        """Try to select attributes title, date and description."""
        try:
            title: str = note[0]
            location, date = note[1].split("|", maxsplit=1)
            description: str = note[3]
        
        except IndexError:
            results: dict = {}
        else:
            results: dict = {
                "date": date,
                "name": title,
                "location": location,
                "description": description
            }
        finally:
            return results

    def create_note(self, attrs: dict):
        self.notes.append(
            KindleNote(
                attrs.get("name"),
                attrs.get("location"),
                attrs.get("description"),
                attrs.get("date"),
            )
        )
        
    def create_all_notes(self, notes: List[dict]) -> None:
        for note in notes:
            if len(note.keys()) == 4:
                self.create_note(note)

class KindleNote:
    """Create a note with proper attributes."""
    
    def __init__(self, name: str, location: str, desc: str, date: str):
        self.name = name
        self.desc = desc
        self.date = date
        self.location = location
        
    def __repr__(self):
        return self.name

In [65]:
app = KindleNoteParser()
text = """
==========
Faktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová)
- Your Highlight on Location 2724-2728 | Added on Wednesday, July 24, 2019 8:41:27 AM

Největším orgánem našeho těla je kůže. Před objevem moderních léků patřila k nejhorším kožním nemocem syfilis. Začínala jako svědivé vřídky a pak si prokousala cestu do kostí, až postihla celou kostru. Nemoc způsobující ohavný vzhled a nesnesitelnou bolest měla v různých zemích různá jména. V Rusku chorobě říkali „polská nemoc“. V Polsku to byla „německá nemoc“, v Německu „francouzská nemoc“ a ve Francii „italská nemoc“. Italové vinu házeli zpátky a nazývali ji „francouzská nemoc“.
==========
Life Is What You Make It (Peter Buffett)
- Your Highlight on Location 233-234 | Added on Monday, August 19, 2019 2:07:48 PM

The problem with honoring the rewards of work rather than the work itself is that the rewards can always be taken away.
==========
"""

splitted = app.process_txt_content(text)
data = app.split_text_into_lines(splitted)
app.create_all_notes(data)
app.notes

[Faktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová),
 Life Is What You Make It (Peter Buffett)]

---